# Homework 2: Word Similarity

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dataset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

### Question 1 (1.0 mark)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://alfonseca.org/eng/research/wordsim353.html">Similarity-353</a>. You need to first obtain this dataset, which is on Canvas (assignment 2). The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on **document frequencies** (not token frequencies) in the Brown corpus, in order to remove rare words. In this homework, **we will be treating the paragraphs of the Brown corpus as our "documents"**. You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized (lemmatize function provided). Store this preprocessed data in *brown_corpus* object (it will be used for question 4 and 5 later).

Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of **$< 8$** in this corpus. You should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*.

Note: the document frequency of a word denotes the number of documents that contains the word.

**Task**: Filter word pairs from *set1.tab* based on document frequencies. Produce *brown_corpus*, which is a list where each element is a set of words for one paragraph (e.g. the first element in *brown_corpus* should contain all the unique word types for the first paragraph). Produce *filtered_gold_standard*, a dictionary of filtered word pairs with human similarity ratings (the dictionary should have (word1, word2) as keys, and similarity ratings as values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *filtered_gold_standard*.

In [245]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

brown_corpus=[]
# lemmatizer
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

###
# Your answer BEGINS HERE
###
'''
1. loadfile
2. produce brown paras: non-alphabetic, lowercased, lemmatized
3. filter
'''
import pandas as pd
import numpy as np
from collections import defaultdict


#1. Load file
df = pd.read_csv('set1.tab', usecols=['Word 1','Word 2', 'Human (mean)' ],  sep='\t')
filtered_gold_standard = df.set_index(['Word 1', 'Word 2'])['Human (mean)'].to_dict()

#2. Produce borwn_corpus
brown_corpus = []
brown_para = brown.paras()
DF = defaultdict(int) 
#print (len(list1[11]))
#print (list1[5])

for para in brown_para:
    unique_word_set = [] #set()
    for sent in para:
        #word_list = []
        for word in set(sent):
            if not word.isalpha():
                continue
            else:
                DF[word] += 1  ## Document frequency
                lemma = lemmatize(word.lower())
                unique_word_set.append(lemma)
        #sent_list.extend(word_list)
    #unique_word_list = list(unique_word_set)        
    brown_corpus.append(unique_word_set)     
    

#3  Calculate document frequency
removable_keys = []

for key in filtered_gold_standard.keys():
    left, right = key

    if DF[left] < 8 or DF[right] < 8:
        removable_keys.append(key)


for key in removable_keys:
    del filtered_gold_standard[key]      
    

###
# Your answer ENDS HERE
###

#print(len(filtered_gold_standard))
#print(filtered_gold_standard)


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\kebin\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kebin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<b>For your testing:</b>

In [246]:
assert(len(brown_corpus)==15667)
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

### Question 2 (1.0 mark)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a **single primary sense**. We define single primary sense here as either: (a) having only one sense (i.e. only one synset), or (b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma.

Note: You should lowercase the lemmas of a synset when matching your word; and if there are multiple lowercased lemmas that match your word, you should sum up the count of all matching lemmas.

Additionally, you should remove any words where the primary sense is **not a noun** (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section.

Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria. When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*.

**Task**: Filter word pairs for any words which do not have a single primary sense and aren't nouns. Produce *final_gold_standard*, a dictionary of filtered word pairs with human similarity ratings. Note: this second filtering is applied on top of the first filtering (question 1). In other words, you shouldn't consider any word pairs that have already been discarded by the first filtering in this question.

**Check**: Use the assertion statements in *"For your testing"* for the expected *final_gold_standard*.

In [247]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}
word_primarysense = {} #a dictionary of (word, primary_sense) (used for next section); primary_sense is a synset

###
# Your answer BEGINS HERE
###
from nltk.corpus import wordnet as wn


def is_primary_noun(word):
    word_synset = wn.synsets(word)    
    primary = False
    
    if len(word_synset) == 1 and word_synset[0].pos() == 'n':
        primary = True
        word_primarysense[word] = word_synset[0]
    else:
        lemma_list = []
        for synset in word_synset:
                
            lemmas = synset.lemma_names()
            for i, lemma in enumerate(lemmas):
                if lemma.lower() == word:
                    lemma_count = synset.lemmas()[i].count()
                    lemma_list.append((lemma_count, synset  ))

        lemma_list = sorted(lemma_list, key=lambda item: item[0], reverse=True)

        #l\emma_list = sorted(lemma_list, reverse=True)

        if len(lemma_list)>1: 
            first = lemma_list[0]
            second =  lemma_list[1]


            if first[0] >= 4*second[0] and first[1].pos() == 'n':
                primary = True
                word_primarysense[word] = first[1]

    return primary   
    

    
for candidate, sim in filtered_gold_standard.items():
    count = 0
    left, right = candidate
    is_primary_first = is_primary_noun(left)    
    is_primary_second = is_primary_noun(right)
    #print(len(filtered_gold_standard))
    #print('second', is_primary_second)
    #break
    
    if is_primary_first and is_primary_second:
        final_gold_standard[candidate] = sim       


###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

25
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('physics', 'chemistry'): 7.35, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('furnace', 'stove'): 8.79, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'Freud'): 8.21, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08}


<b>For your testing:</b>

In [248]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Computing word similiarity with Lin similarity, NPMI and LSA (3 marks)

### Question 3 (1.0 mark)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*.

**Task**: Compute word pair similarity using Lin similarity for the test set. Produce *lin_similarities*, a dictionary of word pairs (keys) and computed Lin similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *lin_similarities*. 

In [249]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

brown_ic = wordnet_ic.ic('ic-brown.dat')
for word_pairs, v in final_gold_standard.items():
    
    first_primary = word_primarysense[word_pairs[0]]
    second_primary = word_primarysense[word_pairs[1]]
    lin_sim = wn.lin_similarity(first_primary, second_primary, brown_ic)

    lin_similarities[word_pairs] = lin_sim
    
###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\kebin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
{('bread', 'butter'): 0.711420490146294, ('professor', 'doctor'): 0.7036526610448273, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.7888839126424345, ('football', 'basketball'): 0.7536025025710653, ('football', 'tennis'): 0.7699955045932811, ('physics', 'chemistry'): 0.817907088214157, ('alcohol', 'chemistry'): 0.062235427146896456, ('baby', 'mother'): 0.6315913189894092, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.6969176573027711, ('coast', 'shore'): 0.9632173804623256, ('furnace', 'stove'): 0.22813808925013807, ('brother', 'monk'): 0.24862817480738675, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.5991131628821826, ('monk', 'slave'): 0.2543108201944307, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'Freud'): -0.0, ('psychology', 'mind'): 0.3

<b>For your testing:</b>

In [250]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

# Question 4 (1.0 mark)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-log_2(p(x,y))} = \frac{log_2(p(x)p(y))}{log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1]. 

You should use the *brown_corpus* object you've created in question 1 as your corpus here. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*.

**Task**: Compute word pair similarity using NPMI similarity for the test set. Produce *NPMI_similarities*, a dictionary of word pairs (keys) and computed NPMI similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *NPMI_similarities*.

In [251]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}
import math
from collections import defaultdict
###
# Your answer BEGINS HERE
###

num_words = 0
co_coocr = defaultdict(int)

num_x_co_occur = 0
num_y_co_occur = 0
num_xy_co_occur = 0
'''
for para in brown_corpus:
    num_words += len(para)
    
    for word_pairs, v in final_gold_standard.items():
        if word_pairs[0] in para:
            co_coocr[word_pairs[0]]=1
            
        if word_pairs[1] in para:
            co_coocr[word_pairs[1]]+=1
                    
        if word_pairs[0] in para and word_pairs[1] in para:
            co_coocr[word_pairs]+=1        
'''

for word_pairs, v in final_gold_standard.items():
    word1, word2 =word_pairs    

    word1_count = 0
    word2_count = 0
    both_count = 0
    total_count = 0.0 # so that division results in a float
    for para in brown_corpus:
        total_count += len(para) 
        
        if word_pairs[0] in para:
            word1_count +=1
                
            if word_pairs[1] in para:
                    both_count += 1

        elif word_pairs[1] in para:
            word2_count += 1

    px = word1_count/total_count
    py = word2_count/total_count
    pxy = both_count/total_count  
    npmi = 0
    
    if pxy>0:
        npmi = (math.log(px*py, 2)/math.log(pxy, 2) ) - 1
    else:
        npmi = -1
    NPMI_similarities[word_pairs] = npmi
             


###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.6741677913144302, ('professor', 'doctor'): -1, ('student', 'professor'): 0.5597532490313666, ('stock', 'egg'): 0.38294333427476546, ('money', 'cash'): 0.46410023934856315, ('football', 'basketball'): 0.7544542184406218, ('football', 'tennis'): -1, ('physics', 'chemistry'): -1, ('alcohol', 'chemistry'): 0.6374063275923236, ('baby', 'mother'): 0.5291939980607001, ('car', 'automobile'): 0.5707416773223375, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.6038831797228301, ('furnace', 'stove'): -1, ('brother', 'monk'): 0.4448483233908309, ('journey', 'car'): -1, ('coast', 'hill'): 0.3529280471525129, ('monk', 'slave'): -1, ('coast', 'forest'): 0.4716275483849135, ('psychology', 'doctor'): 0.4725744655181885, ('psychology', 'Freud'): -1, ('psychology', 'mind'): 0.4580322827619818, ('psychology', 'health'): -1, ('psychology', 'science'): 0.6018482824800955, ('planet', 'moon'): 0.6758289889223563}


In [255]:
for para in brown_corpus:
    if 'Freud' in para:
        print('yes')

<b>For your testing:</b>

In [252]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

### Question 5 (1.0 mark)

**Instructions:** Here we'll use singular value decomposition (SVD) to derive similarity scores using the Latent Semantic Analysis (LSA) method. Recall that LSA applies SVD and truncation to get a dense vector representation of a word type. To measure similarity between two words we calculate cosine similarity between the LSA vectors of the words.

We'll first build a term-document frequency matrix (as before, a document is a paragraph in Brown corpus). That is, the rows corresponds to words in the vocabulary, and the columns represent the paragraphs/documents in Brown corpus. Each cell records the document frequency of a word (1 if the word appears in the document, 0 otherwise). You should use the *brown_corpus* object created in question 1 as your corpus to build the term-document frequency matrix.

Given the term-document frequency matrix, we'll use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for the word pairs in the test set.

You can use the workshop's notebook on distributional similarity (``09-distributional-similarity.ipynb``) as a starting point, but note that we use term-document frequency matrix (as opposed to the tf-idf matrix in the workshop).

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*.

**Task**: Compute word pair similarity using LSA. Produce *LSA_similarities*, a dictionary of word pairs (keys) and computed LSA similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *LSA_similarities*. 



In [256]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np
from numpy.linalg import norm

def dummy(doc):
    return doc

def cos_sim(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))

#for para in brown_corpus:
#    if 'physics' in para:
#        print('physics')
#        break
        


vectorizer = CountVectorizer( tokenizer=dummy, preprocessor=dummy)
brown_matrix_tdf = vectorizer.fit_transform(brown_corpus)
brown_matrix_tdf_transposed = csr_matrix(brown_matrix_tdf).transpose()
svd = TruncatedSVD(n_components=500)

brown_matrix_lowrank = svd.fit_transform(brown_matrix_tdf_transposed)

for word_pairs, v in final_gold_standard.items():
    word1, word2 =word_pairs    
    v1 = brown_matrix_lowrank[vectorizer.vocabulary_[lemmatize(word1.lower())]]
    v2 = brown_matrix_lowrank[vectorizer.vocabulary_[lemmatize(word2.lower())]]

    sim = cos_sim(v1, v2)
    LSA_similarities[word_pairs] = sim


###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('bread', 'butter'): 0.17977048050583863, ('professor', 'doctor'): 0.02887863758677931, ('student', 'professor'): 0.13653636123772742, ('stock', 'egg'): 0.03328209304832002, ('money', 'cash'): 0.14717661223447429, ('football', 'basketball'): 0.18594296537618957, ('football', 'tennis'): 0.07988891931725159, ('physics', 'chemistry'): 0.25101099133609306, ('alcohol', 'chemistry'): 0.2550685676573444, ('baby', 'mother'): 0.18116439670519519, ('car', 'automobile'): 0.2629680454774084, ('journey', 'voyage'): 0.0918898310090328, ('coast', 'shore'): 0.36937733312006565, ('furnace', 'stove'): 0.1910109340111165, ('brother', 'monk'): 0.04309866658238733, ('journey', 'car'): -0.00748404881986118, ('coast', 'hill'): 0.19263070004462618, ('monk', 'slave'): 0.003196066399360036, ('coast', 'forest'): 0.06776628277359624, ('psychology', 'doctor'): 0.04118665936890422, ('psychology', 'Freud'): 0.18977332443167227, ('psychology', 'mind'): 0.09322466015922276, ('psychology', 'health'): -0.03185720827419

<b>For your testing:</b>

In [257]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## Computing word similarity with feedforward language model (3 marks)

### Question 6 (1.0 mark)

**Instructions**: Here we'll build a n-gram neural language model to learn word embeddings, and compute word similarity based on the word embeddings. As before we will use the Brown corpus as training data for our language model, and the first step is to collect the vocabulary.

As before, we'll treat paragraphs in the Brown corpus as our documents. The first 12K paragraphs/documents will serve as our training data, and the rest (3K+ documents) as development data. The first step towards building a language model is to collect the vocabulary, i.e. the set of unique word types in our training data. When collecting the word types, you should lowercase all words, and only keep word types that have a frequency $>= 5$. Store your vocabulary in the _vocab_ object.

Note: 
  - we'll be using _$<$UNK$>$_ to represent unseen words, and so _vocab_ is initialised with the special _$<$UNK$>$_.
  - you do not need to do any other additional preprocessing aside from the aforementioned steps (e.g. no need to remove symbols, etc)
  - you should not use *brown_corpus* here, as you need the words in their original order (*brown_corpus* stores a set of words for each paragraph, and so do not contain word order information)

**Task**: Collect a set of unique word types in the training portion (first 12K paragraphs) of the Brown corpus. Produce _vocab_, which is a set that contains all the word types.

**Check**: Use the assertion statements in *"For your testing"* below to check the vocabulary size.

In [264]:
num_train = 12000
UNK_symbol = "<UNK>"
vocab = set([UNK_symbol])

###
# Your answer BEGINS HERE
###

brown_para = brown.paras()
TF = defaultdict(int) 

for para in brown_para[:num_train]:
    for sent in para:
        for word in sent:
            TF[word.lower()] += 1 
            vocab.add(word.lower())     
    
     
remove_list = [k for k, v in sorted(TF.items(), key=lambda item: item[1]) if v<5 ]
print(len(vocab))
vocab = vocab.difference(remove_list)
    
###
# Your answer ENDS HERE
###

print(len(vocab))

45414
12681


**For your testing:**

In [265]:
assert(len(vocab) > 8000 and len(vocab) < 20000)

### Question 7 (1.0 mark)

**Instructions**: As we'll be building a trigram neural language model (based on lecture 7, page 20), the next step is to collect trigrams to construct our training data. In a trigram neural language model, for example if we have the trigram _cow eats grass_, the input to the model is the first two terms of a trigram (_cow_ and _eats_), and the language model's aim is to predict the last term of the trigram (_grass_). Your task here is to construct the training and development data for the language model. Just like the previous step, the first 12K paragraphs will serve as our training data, and the remaining 3K+ will be for development. You'll need to map words into IDs when constructing the training and development data. Any words that are not in _vocab_ should be mapped to the special _$<$UNK$>$_ symbol.

As an example, given the sentence "_a big fat hungry cow ._", you should create the following training examples:

|input|target|
|:---:|:----:|
|<i>a</i>, _big_|_fat_|
|_big_, _fat_|_hungry_|
|_fat_, _hungry_|_cow_|
|_hungry_, _cow_|_._|


Note:
   - _vocab_ is a set and so does not map words into IDs. You'll need to create a word to ID mapping first based on _vocab_.
   - A trigram should not cross sentence boundary.
   - You should ignore sentences that have less than 3 words (as they are too short to form trigrams).
   - We won't need special starting symbol when collecting the trigrams, as we are only interested in learning word embeddings here (and not computing probabilities of a sentence).


**Task**: Create training and development data. The training input and target data should be stored in the <i>x_train</i> and <i>y_train</i> respectively, and development input and target data in <i>x_dev</i> and <i>y_dev</i> respectively.

**Check**: Use the assertion statements in *"For your testing"* below for the expected shape of the outputs.

In [296]:
###
# Your answer BEGINS HERE
###
mapping = {}
x_train_0 = []
x_train_1 = []
y_train = []
x_dev_0 = []
x_dev_1 = []
y_dev = []

def get_mapping(word):
    if word in mapping:
        return mapping[elem]
    else:
        return mapping['<UNK>']
    
for i, elem in enumerate(vocab):
    mapping[elem] = i


for para in brown_para[:num_train]:
    for sent in para:
        if len(sent)>=3:
            for i, word in enumerate(sent):
                if i+2 == len(sent):
                    break                
                x_train_0.append( get_mapping(sent[i]) )
                x_train_1.append( get_mapping(sent[i+1]) )
                
                y_train.append( get_mapping(sent[i+2]) )
    
for para in brown_para[num_train:]:
    for sent in para:
        if len(sent)>=3:
            for i, word in enumerate(sent):
                if i+2 == len(sent):
                    break                
                x_dev_0.append( get_mapping(sent[i])  )
                x_dev_1.append( get_mapping(sent[i+1]) )
                y_dev.append( get_mapping(sent[i+2]) )      
    
x_train_0 =  np.array(x_train_0) 
x_train_1 =  np.array(x_train_1) 
y_train =  np.array(y_train) 
x_dev_0 =  np.array(x_dev_0) 
x_dev_1 =  np.array(x_dev_1) 
y_dev =  np.array(y_dev) 

###
# Your answer ENDS HERE
###
    
print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)

(872823, 2)
(872823,)
(174016, 2)
(174016,)


**For your testing:**

In [282]:
assert(x_train.shape[0] == y_train.shape[0])
assert(x_dev.shape[0] == y_dev.shape[0])
assert(x_train.shape[0] > 500000)
assert(x_dev.shape[0] > 50000)

### Question 8 (1.0 mark)

**Instructions**: Now let's build the trigram neural language model. We'll use the language model described in lecture 7 (page 20):

$x' = e(x_1) \oplus e(x_2)$

$h = \tanh(W_1 x' + b)$

$y = $ softmax$(W_2 h)$

where $\oplus$ is the concatenation operation, $x_1$ and $x_2$ are the input words, $e$ is an embedding function, and $y$ is the target word. You can use either `keras` or `pytorch` for building your model.

Set the dimension of the word embeddings and $h$ to 100, and train your model with 3 epochs with a batch size of 256. You will not need to tune hyper-parameters for this task.

After the model is trained, use the word embeddings to compute word similarity in the test set. Store the similarity scores in a dictionary called *lm_similarities*.

Note:
  - For words in the test set that are not in your vocabulary, you should treat them as unknown words. In other words, you should use $<$UNK$>$'s embedding to represent these words.
  - The training may take some time on CPU. You can run your notebook on colab with a GPU if you want faster training (see the programming exercise in workshop-07 if you're not familiar with colab)

**Task**: Train a trigram neural language model, and use the learned word embeddings to compute word similarity. Produce *lm_similarities*, a dictionary that contains word pairs as keys and similarity scores as values.

In [304]:
lm_similarities = {}

from keras.models import Sequential, Model
from keras import layers
from keras.layers import Flatten, Dense, Input, Concatenate, Embedding
import tensorflow as tf

vocab_size = len(vocab)
embedding_dim = 100

###
# Your answer BEGINS HERE
###

a = Input(shape=[1])
b = Input(shape=[1])

emb_a = Embedding(input_dim=vocab_size, output_dim=embedding_dim )(a)
emb_b = Embedding(input_dim=vocab_size, output_dim=embedding_dim )(b)

concat = Concatenate()([emb_a, emb_b]) 

flat = Flatten()(concat)

hidden = Dense(100, activation='tanh')(flat)


out = Dense(vocab_size, activation='softmax')(hidden)


model = Model(inputs=[a, b], outputs=[out])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
model.fit([x_train_0, x_train_1 ] , y_train, epochs=3, verbose=True, validation_data=([x_dev_0, x_dev_1], y_dev), batch_size=256)


embeddings = model.get_layer(index=0).get_weights()[0] #word embeddings layer

                                                     
                                                     
for word_pairs, v in final_gold_standard.items():
    word1, word2 =word_pairs    
    v1 = embeddings[ get_mapping(word1) ]
    v2 = embeddings[ get_mapping(word2) ]

    sim = cos_sim(v1, v2)
    lm_similarities[word_pairs] = sim


###
# Your answer ENDS HERE
###


print(lm_similarities)

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_33 (Embedding)        (None, 1, 100)       1268100     input_35[0][0]                   
__________________________________________________________________________________________________
embedding_34 (Embedding)        (None, 1, 100)       1268100     input_36[0][0]                   
___________________________________________________________________________________________

IndexError: list index out of range

## 3. Comparison with the Gold Standard (1 mark)

### Question 9 (1.0 mark)

**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA', 'lm').

**Task**: Compute the Pearson correlation coefficient between the estimated similarity scores (Lin, NPMI and LSA similarities) and the gold standard similarity ratings. Produce *pearson_correlations*, a dictionary containing the methods as keys and correlations as values.

**Check**: Use the assertion statements in *"For your testing"* below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs! 

In [325]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA', 'lm'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###

lin = [v for k, v in lin_similarities.items() ]
npmi = [v for k, v in NPMI_similarities.items() ]
lsa = [v for k, v in LSA_similarities.items() ]
#lm = [v for k, v in lm_similarities.items() ]
gold = [v for k, v in final_gold_standard.items() ]


key_name_list = ['lin', 'NPMI', 'LSA'  ] #'lm'
sim_lists = [lin, npmi, lsa ] #lm

for i, sim in enumerate(sim_lists): 
    coef = pearsonr(sim, gold)[0] 
    key_name = key_name_list[i]
    pearson_correlations[key_name] = coef


###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.4773360428509657, 'NPMI': -0.010366954503909204, 'LSA': 0.4262901234757967}


<b>For your testing:</b>

In [326]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.